## 1.6 Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

In [172]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "Green Taxi Trip Records", we'll use "For-Hire Vehicle Trip Records".

Download the data for January and February 2021.

Note that you need "For-Hire Vehicle Trip Records", not "High Volume For-Hire Vehicle Trip Records".

Read the data for January. How many records are there?

* 1054112
* 1154112
* 1254112
* 1354112

In [173]:
df_jan = pd.read_parquet('data/fhv_tripdata_2021-01.parquet')

In [174]:
print(f'There are {df_jan.shape[0]:,} records')

There are 1,154,112 records



## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the average trip duration in January?

* 15.16
* 19.16
* 24.16
* 29.16

In [175]:
# create field duration of taxi rides in minutes
df_jan['duration'] = (df_jan.dropOff_datetime - df_jan.pickup_datetime).dt.total_seconds().div(60)

In [176]:
print(f'The average trip duration in January is {df_jan.duration.mean():,.3f} minutes.')

The average trip duration in January is 19.167 minutes.



## Data preparation

Check the distribution of the duration variable. There are some outliers. 

Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

How many records did you drop? 

In [177]:
a = ['a', 'b', 'c']
all(x in a for x in ['a', 'b'])

True

In [178]:
def read_dataframe(filename):
    if not filename.endswith('.parquet'):
        Exception('filetype not supported')
    
    df = pd.read_parquet(filename)

    if any(x not in df.columns for x in ['PUlocationID', 'DOlocationID', 'dropOff_datetime', 'pickup_datetime']):
        Exception('columns missing')

    # create field duration of taxi rides in minutes
    df['duration'] = (df.dropOff_datetime - df.pickup_datetime).dt.total_seconds().div(60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].fillna(-1).astype(str)
    
    return df

In [179]:
df_train = read_dataframe('data/fhv_tripdata_2021-01.parquet')

In [180]:
print(f'We have dropped {df_jan.shape[0] - df_train.shape[0]:,} values ({(df_jan.shape[0] - df_train.shape[0])/df_jan.shape[0]*100:.1f}%).')

We have dropped 44,286 values (3.8%).



## Q3. Missing values

The features we'll use for our model are the pickup and dropoff location IDs. 

But they have a lot of missing values there. Let's replace them with "-1".

What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.

* 53%
* 63%
* 73%
* 83%

In [181]:
df_train.PUlocationID.eq('-1.0').value_counts(normalize=True).mul(100)[1].round(1).astype(str)+'%'

'83.5%'


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix? (The number of columns).

* 2
* 152
* 352
* 525
* 725

In [182]:
train_dicts = df_train[['PUlocationID', 'DOlocationID']].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [183]:
print(f'The matrix has {X_train.shape[1]} columns.')

The matrix has 525 columns.



## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 5.52
* 10.52
* 15.52
* 20.52



In [184]:
target = 'duration'
y_train = df_train[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519107204616


## Q6. Evaluating the model

Now let's apply this model to the validation dataset (Feb 2021). 

What's the RMSE on validation?

* 6.01
* 11.01
* 16.01
* 21.01

In [185]:
df_test = read_dataframe('data/fhv_tripdata_2021-02.parquet')

In [186]:
test_dicts = df_test[['PUlocationID', 'DOlocationID']].to_dict(orient='records')

X_test = dv.transform(test_dicts)

target = 'duration'
y_test = df_test[target].values

y_pred = lr.predict(X_test)

mean_squared_error(y_test, y_pred, squared=False)

11.01428315568312